In [1]:
import pandas as pd
import requests
import h3
import urllib

In [2]:
df=pd.read_pickle('businessesforsale.detail.pkl')

In [3]:
df.head()

,props,broker_details,listing_id,listing_id_text,listing_title,address,asking_price,sales_revenue,cash_flow,listing,business_operation,property-information,ranchise-terms,other_information
0,{'og:url': 'https://us.businessesforsale.com/u...,Transworld Business Advisors Greenville East,3055843,Listing ID: 3055843\nSeller ref: 52661-814795,"Grab This Profitable, High Traffic Mexican Bar...","South Carolina, US","$847,880 \nFurniture / Fixtures and Inventory ...","$1,984,186","$252,751",Jump on this growth opportunity that produced ...,{'Reasons for selling:': 'other business inter...,"{'Location:': 'Beautiful dining room, bar, kit...",{},"{'Support & training:': '2 weeks', 'Owner fina..."
1,{'og:url': 'https://us.businessesforsale.com/u...,Empire Business Management,3399695,Listing ID: 3399695\nSeller ref: 28646,Eco Friendly Car Wash On Mixed Use Property Fo...,"Massachusetts, US","$1,500,000","$250,000","$174,000",HS Listing ID-28646\n\n\nEco-Friendly Semi-Abs...,{'Expansion Potential:': 'There is a new Sobri...,"{'Real Estate:': 'Real Property Included', 'Pr...",{},{'Support & training:': 'Yes.'}
2,{'og:url': 'https://us.businessesforsale.com/u...,Benchmark International,2986000,Listing ID: 2986000\nSeller ref: BM001492,Established And Full Size Electrical Contracto...,US,On request,Over $5M,$500K - $2.5M,The company is a non-union electrical contract...,{'Reasons for selling:': 'The owners are appro...,{'Premises Details:': 'The property contains o...,{},{'Related Documents:': 'Document'}
3,{'og:url': 'https://us.businessesforsale.com/u...,Vested Business Brokers HQ,3322603,Listing ID: 3322603\nSeller ref: 22891,Auto Repair Business In Middlesex County For Sale,"Middlesex County,\nNew Jersey, US","$135,000 \nInventory / Stock included","$63,200","$54,800",One of a kind opportunity to own both business...,{'Reasons for selling:': 'Retiring after 37 ye...,{},{},"{'Support & training:': 'Seller will train', '..."
4,{'og:url': 'https://us.businessesforsale.com/u...,BTI Group,3333333,Listing ID: 3333333\nSeller ref: EG10926,Popular Profitable Garden Center With Inventor...,"Lane County,\nOregon, US","$299,000 \nInventory / Stock included","$513,643","$48,280","Popular, profitable and dependable local sourc...","{'Reasons for selling:': 'Nursing School', 'Tr...","{'Real Estate:': 'Lease', 'Lease Terms:': 'The...",{},"{'Support & training:': '4 weeks, 20 hours er ..."


In [10]:
df=df.loc[:1]

In [13]:
def geocode(location):
    location=' '.join(location.split())
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(location) +'?format=json'
    #url='http://localhost:9080/search.php?q='+urllib.parse.quote(location) 
    result=requests.get(url)
    if(result.status_code==200):
        response = result.json()
        #print(response)
        if len(response):
            return( 1, response[0]['lat'], response[0]['lon'], result.status_code)
    return (0,  result.status_code)

In [14]:
%%time
df['geocode'] = df['address'].apply(geocode)

CPU times: user 40.1 ms, sys: 70.6 ms, total: 111 ms
Wall time: 7.54 s


In [15]:
def h3_15_idx(geocode):
    if geocode[0]:
        return h3.geo_to_h3(float(geocode[1]),float(geocode[2]),15)
    return pd.NA

In [16]:
df['h3_15'] = df['geocode'].apply(h3_15_idx)

In [17]:
df.head()

,props,broker_details,listing_id,listing_id_text,listing_title,address,asking_price,sales_revenue,cash_flow,listing,business_operation,property-information,ranchise-terms,other_information,geocode,h3_15
0,{'og:url': 'https://us.businessesforsale.com/u...,Transworld Business Advisors Greenville East,3055843,Listing ID: 3055843\nSeller ref: 52661-814795,"Grab This Profitable, High Traffic Mexican Bar...","South Carolina, US","$847,880 \nFurniture / Fixtures and Inventory ...","$1,984,186","$252,751",Jump on this growth opportunity that produced ...,{'Reasons for selling:': 'other business inter...,"{'Location:': 'Beautiful dining room, bar, kit...",{},"{'Support & training:': '2 weeks', 'Owner fina...","(1, 33.6874388, -80.4363743, 200)",8f44d1692588552
1,{'og:url': 'https://us.businessesforsale.com/u...,Empire Business Management,3399695,Listing ID: 3399695\nSeller ref: 28646,Eco Friendly Car Wash On Mixed Use Property Fo...,"Massachusetts, US","$1,500,000","$250,000","$174,000",HS Listing ID-28646\n\n\nEco-Friendly Semi-Abs...,{'Expansion Potential:': 'There is a new Sobri...,"{'Real Estate:': 'Real Property Included', 'Pr...",{},{'Support & training:': 'Yes.'},"(1, 42.3788774, -72.032366, 200)",8f2a32372a8b69d


In [10]:
#df.to_pickle('businessesforsale.detail.h3.pkl')

In [ ]:
# note may need to install local instance
# https://github.com/mediagis/nominatim-docker

In [20]:
print(dict(df.loc[0]))

{'props': {'og:url': 'https://us.businessesforsale.com/us/grab-this-profitable-high-traffic-mexican-bar-for-sale.aspx', 'og:type': 'website', 'og:title': 'Buy a grab this profitable, high traffic mexican bar', 'og:description': 'Jump on this growth opportunity that produced $1.6M 2020 sales, projecting $2M+ in 2021! Incredible location with strong brand, great food, a loyal…', 'og:image': 'https://us.businessesforsale.com/uploads/ecfb3c62-ec55-4c29-bd2d-adba011db616.jpg?zobw1tYWteEDWa+FWofdlA=='}, 'broker_details': 'Transworld Business Advisors Greenville East', 'listing_id': 3055843, 'listing_id_text': 'Listing ID: 3055843\nSeller ref: 52661-814795', 'listing_title': 'Grab This Profitable, High Traffic Mexican Bar For Sale', 'address': 'South Carolina, US', 'asking_price': '$847,880 \nFurniture / Fixtures and Inventory / Stock included', 'sales_revenue': '$1,984,186', 'cash_flow': '$252,751', 'listing': "Jump on this growth opportunity that produced $1.6M 2020 sales, projecting $2M+ i